In [206]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Factory Data
fc = ['Cleveland', 'Austin']
fc_range = len(fc)
f_dc = [[.5, .5, 1, .2],
        [999, .3, .5, .2]]
f_cust = [[1, 999, 1.5, 2, 999, 999],
          [2, 999, 999, 999, 999, 999]]

f_prod = [150000, 200000]

# Distribution Centers Data
dc = ['Pittsburgh', 'Boulder', 'DesMoines', 'Tucson']
dc_r = len(dc)
dc_cust = [[999, 1.5, .5, 1.5, 999, 1],
        [1, .5, .5, 1, .5, 999],
        [999, 1.5, 2, 999, 0.5, 1.5],
        [999, 999, .2, 1.5, .5, 1.5]]

dc_max = [70000, 50000, 100000, 40000]

# Customer data
cust = ['Cust_1', 'Cust_2', 'Cust_3', 'Cust_4', 'Cust_5', 'Cust_6']
cust_range = len(cust)
cust_demand = [50000, 10000, 40000, 35000, 60000, 20000]

In [207]:

m = gp.Model('Distribution') # insert model name in quotes

m.ModelSense = GRB.MINIMIZE

x = []
for i in fc:
    inner_list = []
    for j in dc:
        inner_list.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{j}'))
    x.append(inner_list)


y = []
for i in fc:
    inner_list1 = []
    for k in cust:
        inner_list1.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{k}'))
    y.append(inner_list1)



z = []
for j in dc:
    inner_list2 = []
    for k in cust:
        inner_list2.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{j}_to_{k}'))
    z.append(inner_list2)

m.update()
m.display()

Minimize
  0.0
Subject To


C:\Users\samha\AppData\Local\Temp\ipykernel_24080\1878312230.py:30: DeprecationWarning: Model.display() is deprecated
  m.display()


In [208]:
''' Create objective function and update model '''
m.setObjective((gp.quicksum(f_dc[i][j]*x[i][j] for j in range(len(dc)) for i in range(len(fc)))) + 
               (gp.quicksum(f_cust[i][k]*y[i][k] for i in range(len(fc)) for k in range(len(cust)))) +
               (gp.quicksum(dc_cust[j][k]*z[j][k] for k in range(len(cust_try2)) for j in range(len(dc_cust)))))

m.update()
m.display()

Minimize
0.5 Cleveland_to_Pittsburgh + 0.5 Cleveland_to_Boulder + Cleveland_to_DesMoines
+ 0.2 Cleveland_to_Tucson + 999.0 Austin_to_Pittsburgh + 0.3 Austin_to_Boulder
+ 0.5 Austin_to_DesMoines + 0.2 Austin_to_Tucson + Cleveland_to_Cust_1
+ 999.0 Cleveland_to_Cust_2 + 1.5 Cleveland_to_Cust_3 + 2.0 Cleveland_to_Cust_4
+ 999.0 Cleveland_to_Cust_5 + 999.0 Cleveland_to_Cust_6 + 2.0 Austin_to_Cust_1
+ 999.0 Austin_to_Cust_2 + 999.0 Austin_to_Cust_3 + 999.0 Austin_to_Cust_4
+ 999.0 Austin_to_Cust_5 + 999.0 Austin_to_Cust_6 + 999.0 Pittsburgh_to_Cust_1
+ 1.5 Pittsburgh_to_Cust_2 + 0.5 Pittsburgh_to_Cust_3 + 1.5 Pittsburgh_to_Cust_4
+ 999.0 Pittsburgh_to_Cust_5 + Pittsburgh_to_Cust_6 + Boulder_to_Cust_1
+ 0.5 Boulder_to_Cust_2 + 0.5 Boulder_to_Cust_3 + Boulder_to_Cust_4
+ 0.5 Boulder_to_Cust_5 + 999.0 Boulder_to_Cust_6 + 999.0 DesMoines_to_Cust_1
+ 1.5 DesMoines_to_Cust_2 + 2.0 DesMoines_to_Cust_3 + 999.0 DesMoines_to_Cust_4
+ 0.5 DesMoines_to_Cust_5 + 1.5 DesMoines_to_Cust_6 + 999.0 Tucson_to

C:\Users\samha\AppData\Local\Temp\ipykernel_24080\265499519.py:7: DeprecationWarning: Model.display() is deprecated
  m.display()


In [209]:

for i in range(len(fc)):
    m.addConstr((gp.quicksum(x[i][j] for j in range(len(dc))) + 
                 gp.quicksum(y[i][k] for k in range(len(cust))) <= f_prod[i]),
                name='StayUnderCapacity')

for j in range(len(dc)):
    m.addConstr((gp.quicksum(x[i][j] for i in range(len(fc))) <= dc_max[j]),
                name='MaxInput')
###
for j in range(len(dc)):
    m.addConstr((gp.quicksum(z[j][k] for k in range(len(cust))) == 
                 gp.quicksum(x[i][j] for i in range(len(fc)))),
               name='QuantityOutOfDC')

for k in range(len(cust)):
    m.addConstr((gp.quicksum(y[i][k] for i in range(len(fc))) + 
                 gp.quicksum(z[j][k] for j in range(len(dc))) == cust_demand[k]),
                name='CustomerDemand')


m.update()

m.display()

Minimize
0.5 Cleveland_to_Pittsburgh + 0.5 Cleveland_to_Boulder + Cleveland_to_DesMoines
+ 0.2 Cleveland_to_Tucson + 999.0 Austin_to_Pittsburgh + 0.3 Austin_to_Boulder
+ 0.5 Austin_to_DesMoines + 0.2 Austin_to_Tucson + Cleveland_to_Cust_1
+ 999.0 Cleveland_to_Cust_2 + 1.5 Cleveland_to_Cust_3 + 2.0 Cleveland_to_Cust_4
+ 999.0 Cleveland_to_Cust_5 + 999.0 Cleveland_to_Cust_6 + 2.0 Austin_to_Cust_1
+ 999.0 Austin_to_Cust_2 + 999.0 Austin_to_Cust_3 + 999.0 Austin_to_Cust_4
+ 999.0 Austin_to_Cust_5 + 999.0 Austin_to_Cust_6 + 999.0 Pittsburgh_to_Cust_1
+ 1.5 Pittsburgh_to_Cust_2 + 0.5 Pittsburgh_to_Cust_3 + 1.5 Pittsburgh_to_Cust_4
+ 999.0 Pittsburgh_to_Cust_5 + Pittsburgh_to_Cust_6 + Boulder_to_Cust_1
+ 0.5 Boulder_to_Cust_2 + 0.5 Boulder_to_Cust_3 + Boulder_to_Cust_4
+ 0.5 Boulder_to_Cust_5 + 999.0 Boulder_to_Cust_6 + 999.0 DesMoines_to_Cust_1
+ 1.5 DesMoines_to_Cust_2 + 2.0 DesMoines_to_Cust_3 + 999.0 DesMoines_to_Cust_4
+ 0.5 DesMoines_to_Cust_5 + 1.5 DesMoines_to_Cust_6 + 999.0 Tucson_to

C:\Users\samha\AppData\Local\Temp\ipykernel_24080\459569856.py:23: DeprecationWarning: Model.display() is deprecated
  m.display()


In [210]:


# ''' Optimize model '''
m.optimize()

for v in m.getVars():
   if v.X > 0:
       print(f'{v.varName}: {v.X}')

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0x8adfdc32
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.02s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.650000e+05   0.000000e+00      0s
       7    2.0850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.085000000e+05
Cleveland_to_Pittsburgh: 20000.0
Cleveland_to_Tucson: 40000.0
Austin_to_Boulder: 50000.0
Austin_to_DesMoines: 55000.0
Cleveland_to_Cust_1: 50000.0
Pittsburg

## Effect of Increasing the Factory Capacity

In [184]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Factory Data
fc = ['Cleveland', 'Austin']
fc_range = len(fc)
f_dc = [[.5, .5, 1, .2],
        [999, .3, .5, .2]]
f_cust = [[1, 999, 1.5, 2, 999, 999],
          [2, 999, 999, 999, 999, 999]]

f_prod = [150000, 200000]

# Distribution Centers Data
dc = ['Pittsburgh', 'Boulder', 'DesMoines', 'Tucson']
dc_r = len(dc)
dc_cust = [[999, 1.5, .5, 1.5, 999, 1],
        [1, .5, .5, 1, .5, 999],
        [999, 1.5, 2, 999, 0.5, 1.5],
        [999, 999, .2, 1.5, .5, 1.5]]

dc_max = [70000, 50000, 100000, 40000]

# Customer data
cust = ['Cust_1', 'Cust_2', 'Cust_3', 'Cust_4', 'Cust_5', 'Cust_6']
cust_range = len(cust)
cust_demand = [50000, 10000, 40000, 35000, 60000, 20000]


################

m = gp.Model('Distribution') # insert model name in quotes

m.ModelSense = GRB.MINIMIZE

x = []
for i in fc:
    inner_list = []
    for j in dc:
        inner_list.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{j}'))
    x.append(inner_list)


y = []
for i in fc:
    inner_list1 = []
    for k in cust:
        inner_list1.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{k}'))
    y.append(inner_list1)



z = []
for j in dc:
    inner_list2 = []
    for k in cust:
        inner_list2.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{j}_to_{k}'))
    z.append(inner_list2)

m.update()
# m.display()

####################

m.setObjective((gp.quicksum(f_dc[i][j]*x[i][j] for j in range(len(dc)) for i in range(len(fc)))) + 
               (gp.quicksum(f_cust[i][k]*y[i][k] for i in range(len(fc)) for k in range(len(cust)))) +
               (gp.quicksum(dc_cust[j][k]*z[j][k] for k in range(len(cust_try2)) for j in range(len(dc_cust)))))

m.update()
# m.display()

####################

for i in range(len(fc)):
    m.addConstr((gp.quicksum(x[i][j] for j in range(len(dc))) + 
                 gp.quicksum(y[i][k] for k in range(len(cust))) <= f_prod[i]),
                name='StayUnderCapacity')

for j in range(len(dc)):
    m.addConstr((gp.quicksum(x[i][j] for i in range(len(fc))) <= dc_max[j]),
                name='MaxInput')
###
for j in range(len(dc)):
    m.addConstr((gp.quicksum(z[j][k] for k in range(len(cust))) == 
                 gp.quicksum(x[i][j] for i in range(len(fc)))),
               name='QuantityOutOfDC')

for k in range(len(cust)):
    m.addConstr((gp.quicksum(y[i][k] for i in range(len(fc))) + 
                 gp.quicksum(z[j][k] for j in range(len(dc))) == cust_demand[k]),
                name='CustomerDemand')


m.update()

# m.display()

#####################

m.optimize()

for v in m.getVars():
   if v.X > 0:
       print(f'{v.varName}: {v.X}')


#####################
# Get the reduced cost and range of optimality for each variable
for v in m.getVars():
    print(f'{v.VarName} has a reduced cost of {v.RC}')
    print(f'   and a range of optimality from {v.SAObjLow} to {v.SAObjUp}')


# Get the shadow price and the range of feasibility for each constraint
for c in m.getConstrs():
    print(f'{c.constrName} has a shadow price of {c.pi}')
    print(f'   and a range of feasibility from {c.SARHSLow} to {c.SARHSUp}')



Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0x8adfdc32
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.00s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.650000e+05   0.000000e+00      0s
       7    2.0850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.085000000e+05
Cleveland_to_Pittsburgh: 20000.0
Cleveland_to_Tucson: 40000.0
Austin_to_Boulder: 50000.0
Austin_to_DesMoines: 55000.0
Cleveland_to_Cust_1: 50000.0
Pittsburg

## Effect of Increasing Distribution Center Capacity

In [212]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Factory Data
fc = ['Cleveland', 'Austin']
fc_range = len(fc)
f_dc = [[.5, .5, 1, .2],
        [999, .3, .5, .2]]
f_cust = [[1, 999, 1.5, 2, 999, 999],
          [2, 999, 999, 999, 999, 999]]

f_prod = [150000, 200000]

# Distribution Centers Data
dc = ['Pittsburgh', 'Boulder', 'DesMoines', 'Tucson']
dc_r = len(dc)
dc_cust = [[999, 1.5, .5, 1.5, 999, 1],
        [1, .5, .5, 1, .5, 999],
        [999, 1.5, 2, 999, 0.5, 1.5],
        [999, 999, .2, 1.5, .5, 1.5]]

dc_max = [70000, 50000, 100000, 40000]

# Customer data
cust = ['Cust_1', 'Cust_2', 'Cust_3', 'Cust_4', 'Cust_5', 'Cust_6']
cust_range = len(cust)
cust_demand = [50000, 10000, 40000, 35000, 60000, 20000]


################

m = gp.Model('Distribution') # insert model name in quotes

m.ModelSense = GRB.MINIMIZE

x = []
for i in fc:
    inner_list = []
    for j in dc:
        inner_list.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{j}'))
    x.append(inner_list)


y = []
for i in fc:
    inner_list1 = []
    for k in cust:
        inner_list1.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{k}'))
    y.append(inner_list1)



z = []
for j in dc:
    inner_list2 = []
    for k in cust:
        inner_list2.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{j}_to_{k}'))
    z.append(inner_list2)

m.update()
# m.display()

####################

m.setObjective((gp.quicksum(f_dc[i][j]*x[i][j] for j in range(len(dc)) for i in range(len(fc)))) + 
               (gp.quicksum(f_cust[i][k]*y[i][k] for i in range(len(fc)) for k in range(len(cust)))) +
               (gp.quicksum(dc_cust[j][k]*z[j][k] for k in range(len(cust_try2)) for j in range(len(dc_cust)))))

m.update()
# m.display()

####################

for i in range(len(fc)):
    m.addConstr((gp.quicksum(x[i][j] for j in range(len(dc))) + 
                 gp.quicksum(y[i][k] for k in range(len(cust))) <= f_prod[i]),
                name='StayUnderCapacity')

for j in range(len(dc)):
    m.addConstr((gp.quicksum(x[i][j] for i in range(len(fc))) <= dc_max[j]),
                name='MaxInput')
###
for j in range(len(dc)):
    m.addConstr((gp.quicksum(z[j][k] for k in range(len(cust))) == 
                 gp.quicksum(x[i][j] for i in range(len(fc)))),
               name='QuantityOutOfDC')

for k in range(len(cust)):
    m.addConstr((gp.quicksum(y[i][k] for i in range(len(fc))) + 
                 gp.quicksum(z[j][k] for j in range(len(dc))) == cust_demand[k]),
                name='CustomerDemand')


m.update()

# m.display()

#####################

m.optimize()

for v in m.getVars():
   if v.X > 0:
       print(f'{v.varName}: {v.X}')


#####################
# Get the reduced cost and range of optimality for each variable
for v in m.getVars():
    print(f'{v.VarName} has a reduced cost of {v.RC}')
    print(f'   and a range of optimality from {v.SAObjLow} to {v.SAObjUp}')


# Get the shadow price and the range of feasibility for each constraint
for c in m.getConstrs():
    print(f'{c.constrName} has a shadow price of {c.pi}')
    print(f'   and a range of feasibility from {c.SARHSLow} to {c.SARHSUp}')

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0x8adfdc32
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.00s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.650000e+05   0.000000e+00      0s
       7    2.0850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.085000000e+05
Cleveland_to_Pittsburgh: 20000.0
Cleveland_to_Tucson: 40000.0
Austin_to_Boulder: 50000.0
Austin_to_DesMoines: 55000.0
Cleveland_to_Cust_1: 50000.0
Pittsburg

## Changing Cleveland to Customer 1 to $1.25

In [204]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Factory Data
fc = ['Cleveland', 'Austin']
fc_range = len(fc)
f_dc = [[.5, .5, 1, .2],
        [999, .3, .5, .2]]
f_cust = [[1.25, 999, 1.5, 2, 999, 999],
          [2, 999, 999, 999, 999, 999]]

f_prod = [150000, 200000]

# Distribution Centers Data
dc = ['Pittsburgh', 'Boulder', 'DesMoines', 'Tucson']
dc_r = len(dc)
dc_cust = [[999, 1.5, .5, 1.5, 999, 1],
        [1, .5, .5, 1, .5, 999],
        [999, 1.5, 2, 999, 0.5, 1.5],
        [999, 999, .2, 1.5, .5, 1.5]]

dc_max = [70000, 50000, 100000, 40000]

# Customer data
cust = ['Cust_1', 'Cust_2', 'Cust_3', 'Cust_4', 'Cust_5', 'Cust_6']
cust_range = len(cust)
cust_demand = [50000, 10000, 40000, 35000, 60000, 20000]


################

m = gp.Model('Distribution') # insert model name in quotes

m.ModelSense = GRB.MINIMIZE

x = []
for i in fc:
    inner_list = []
    for j in dc:
        inner_list.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{j}'))
    x.append(inner_list)


y = []
for i in fc:
    inner_list1 = []
    for k in cust:
        inner_list1.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{k}'))
    y.append(inner_list1)



z = []
for j in dc:
    inner_list2 = []
    for k in cust:
        inner_list2.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{j}_to_{k}'))
    z.append(inner_list2)

m.update()
# m.display()

####################

m.setObjective((gp.quicksum(f_dc[i][j]*x[i][j] for j in range(len(dc)) for i in range(len(fc)))) + 
               (gp.quicksum(f_cust[i][k]*y[i][k] for i in range(len(fc)) for k in range(len(cust)))) +
               (gp.quicksum(dc_cust[j][k]*z[j][k] for k in range(len(cust_try2)) for j in range(len(dc_cust)))))

m.update()
# m.display()

####################

for i in range(len(fc)):
    m.addConstr((gp.quicksum(x[i][j] for j in range(len(dc))) + 
                 gp.quicksum(y[i][k] for k in range(len(cust))) <= f_prod[i]),
                name='StayUnderCapacity')

for j in range(len(dc)):
    m.addConstr((gp.quicksum(x[i][j] for i in range(len(fc))) <= dc_max[j]),
                name='MaxInput')
###
for j in range(len(dc)):
    m.addConstr((gp.quicksum(z[j][k] for k in range(len(cust))) == 
                 gp.quicksum(x[i][j] for i in range(len(fc)))),
               name='QuantityOutOfDC')

for k in range(len(cust)):
    m.addConstr((gp.quicksum(y[i][k] for i in range(len(fc))) + 
                 gp.quicksum(z[j][k] for j in range(len(dc))) == cust_demand[k]),
                name='CustomerDemand')


m.update()

# m.display()

#####################

m.optimize()

for v in m.getVars():
   if v.X > 0:
       print(f'{v.varName}: {v.X}')


#####################
# Get the reduced cost and range of optimality for each variable
for v in m.getVars():
    print(f'{v.VarName} has a reduced cost of {v.RC}')
    print(f'   and a range of optimality from {v.SAObjLow} to {v.SAObjUp}')


# Get the shadow price and the range of feasibility for each constraint
for c in m.getConstrs():
    print(f'{c.constrName} has a shadow price of {c.pi}')
    print(f'   and a range of feasibility from {c.SARHSLow} to {c.SARHSUp}')

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0x1c8eeac9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.00s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   2.150000e+05   0.000000e+00      0s
       8    2.2100000e+05   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.210000000e+05
Cleveland_to_Pittsburgh: 20000.0
Cleveland_to_Tucson: 40000.0
Austin_to_Boulder: 50000.0
Austin_to_DesMoines: 55000.0
Cleveland_to_Cust_1: 50000.0
Pittsburg

## Cleveland to Customer 1 - $1.60

In [205]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Factory Data
fc = ['Cleveland', 'Austin']
fc_range = len(fc)
f_dc = [[.5, .5, 1, .2],
        [999, .3, .5, .2]]
f_cust = [[1.6, 999, 1.5, 2, 999, 999],
          [2, 999, 999, 999, 999, 999]]

f_prod = [150000, 200000]

# Distribution Centers Data
dc = ['Pittsburgh', 'Boulder', 'DesMoines', 'Tucson']
dc_r = len(dc)
dc_cust = [[999, 1.5, .5, 1.5, 999, 1],
        [1, .5, .5, 1, .5, 999],
        [999, 1.5, 2, 999, 0.5, 1.5],
        [999, 999, .2, 1.5, .5, 1.5]]

dc_max = [70000, 50000, 100000, 40000]

# Customer data
cust = ['Cust_1', 'Cust_2', 'Cust_3', 'Cust_4', 'Cust_5', 'Cust_6']
cust_range = len(cust)
cust_demand = [50000, 10000, 40000, 35000, 60000, 20000]


################

m = gp.Model('Distribution') # insert model name in quotes

m.ModelSense = GRB.MINIMIZE

x = []
for i in fc:
    inner_list = []
    for j in dc:
        inner_list.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{j}'))
    x.append(inner_list)


y = []
for i in fc:
    inner_list1 = []
    for k in cust:
        inner_list1.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{k}'))
    y.append(inner_list1)



z = []
for j in dc:
    inner_list2 = []
    for k in cust:
        inner_list2.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{j}_to_{k}'))
    z.append(inner_list2)

m.update()
# m.display()

####################

m.setObjective((gp.quicksum(f_dc[i][j]*x[i][j] for j in range(len(dc)) for i in range(len(fc)))) + 
               (gp.quicksum(f_cust[i][k]*y[i][k] for i in range(len(fc)) for k in range(len(cust)))) +
               (gp.quicksum(dc_cust[j][k]*z[j][k] for k in range(len(cust_try2)) for j in range(len(dc_cust)))))

m.update()
# m.display()

####################

for i in range(len(fc)):
    m.addConstr((gp.quicksum(x[i][j] for j in range(len(dc))) + 
                 gp.quicksum(y[i][k] for k in range(len(cust))) <= f_prod[i]),
                name='StayUnderCapacity')

for j in range(len(dc)):
    m.addConstr((gp.quicksum(x[i][j] for i in range(len(fc))) <= dc_max[j]),
                name='MaxInput')
###
for j in range(len(dc)):
    m.addConstr((gp.quicksum(z[j][k] for k in range(len(cust))) == 
                 gp.quicksum(x[i][j] for i in range(len(fc)))),
               name='QuantityOutOfDC')

for k in range(len(cust)):
    m.addConstr((gp.quicksum(y[i][k] for i in range(len(fc))) + 
                 gp.quicksum(z[j][k] for j in range(len(dc))) == cust_demand[k]),
                name='CustomerDemand')


m.update()

# m.display()

#####################

m.optimize()

for v in m.getVars():
   if v.X > 0:
       print(f'{v.varName}: {v.X}')


#####################
# Get the reduced cost and range of optimality for each variable
for v in m.getVars():
    print(f'{v.VarName} has a reduced cost of {v.RC}')
    print(f'   and a range of optimality from {v.SAObjLow} to {v.SAObjUp}')


# Get the shadow price and the range of feasibility for each constraint
for c in m.getConstrs():
    print(f'{c.constrName} has a shadow price of {c.pi}')
    print(f'   and a range of feasibility from {c.SARHSLow} to {c.SARHSUp}')

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0xe6421336
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.00s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   2.150000e+05   0.000000e+00      0s
       8    2.3800000e+05   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.380000000e+05
Cleveland_to_Pittsburgh: 20000.0
Cleveland_to_Tucson: 40000.0
Austin_to_Boulder: 50000.0
Austin_to_DesMoines: 60000.0
Cleveland_to_Cust_1: 45000.0
Pittsburg

## Preferences

In [217]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Factory Data
fc = ['Cleveland', 'Austin']
fc_range = len(fc)
f_dc = [[.5, .5, 1, .2],
        [999, .3, .5, .2]]
f_cust = [[1, 999, 1.5, 2, 999, 999],
          [2, 999, 999, 999, 999, 999]]

f_prod = [150000, 200000]

# Distribution Centers Data
dc = ['Pittsburgh', 'Boulder', 'DesMoines', 'Tucson']
dc_r = len(dc)
dc_cust = [[999, 1.5, .5, 1.5, 999, 1],
        [1, .5, .5, 1, .5, 999],
        [999, 1.5, 2, 999, 0.5, 1.5],
        [999, 999, .2, 1.5, .5, 1.5]]

dc_max = [70000, 50000, 100000, 40000]

# Customer data
cust = ['Cust_1', 'Cust_2', 'Cust_3', 'Cust_4', 'Cust_5', 'Cust_6']
cust_range = len(cust)
cust_demand = [50000, 10000, 40000, 35000, 60000, 20000]


################

m = gp.Model('Distribution') # insert model name in quotes

m.ModelSense = GRB.MINIMIZE

x = []
for i in fc:
    inner_list = []
    for j in dc:
        inner_list.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{j}'))
    x.append(inner_list)


y = []
for i in fc:
    inner_list1 = []
    for k in cust:
        inner_list1.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{i}_to_{k}'))
    y.append(inner_list1)



z = []
for j in dc:
    inner_list2 = []
    for k in cust:
        inner_list2.append(m.addVar(vtype = GRB.CONTINUOUS, name = f'{j}_to_{k}'))
    z.append(inner_list2)

m.update()
# m.display()

####################

m.setObjective((gp.quicksum(f_dc[i][j]*x[i][j] for j in range(len(dc)) for i in range(len(fc)))) + 
               (gp.quicksum(f_cust[i][k]*y[i][k] for i in range(len(fc)) for k in range(len(cust)))) +
               (gp.quicksum(dc_cust[j][k]*z[j][k] for k in range(len(cust_try2)) for j in range(len(dc_cust)))))

m.update()
# m.display()

####################

for i in range(len(fc)):
    m.addConstr((gp.quicksum(x[i][j] for j in range(len(dc))) + 
                 gp.quicksum(y[i][k] for k in range(len(cust))) <= f_prod[i]),
                name='StayUnderCapacity')

for j in range(len(dc)):
    m.addConstr((gp.quicksum(x[i][j] for i in range(len(fc))) <= dc_max[j]),
                name='MaxInput')
###
for j in range(len(dc)):
    m.addConstr((gp.quicksum(z[j][k] for k in range(len(cust))) == 
                 gp.quicksum(x[i][j] for i in range(len(fc)))),
               name='QuantityOutOfDC')

for k in range(len(cust)):
    m.addConstr((gp.quicksum(y[i][k] for i in range(len(fc))) + 
                 gp.quicksum(z[j][k] for j in range(len(dc))) == cust_demand[k]),
                name='CustomerDemand')

# Preferences
for k in range(len(cust)):
    if k == 'Cust_2':
        m.addConstr(y[i][k] >= 1, name = 'Cust2_Pittsburgh')


for k in range(len(cust)):
    if k == 'Cust_6':
        m.addConstr(z[j][k] >= 1, name = 'Cust6_DM&Tuscon')




m.update()

# m.display()

#####################

m.optimize()

for v in m.getVars():
   if v.X > 0:
       print(f'{v.varName}: {v.X}')


#####################
# Get the reduced cost and range of optimality for each variable
for v in m.getVars():
    print(f'{v.VarName} has a reduced cost of {v.RC}')
    print(f'   and a range of optimality from {v.SAObjLow} to {v.SAObjUp}')


# Get the shadow price and the range of feasibility for each constraint
for c in m.getConstrs():
    print(f'{c.constrName} has a shadow price of {c.pi}')
    print(f'   and a range of feasibility from {c.SARHSLow} to {c.SARHSUp}')

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0x8adfdc32
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.02s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.650000e+05   0.000000e+00      0s
       7    2.0850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.085000000e+05
Cleveland_to_Pittsburgh: 20000.0
Cleveland_to_Tucson: 40000.0
Austin_to_Boulder: 50000.0
Austin_to_DesMoines: 55000.0
Cleveland_to_Cust_1: 50000.0
Pittsburg